# Nature Conservancy -  Fish Detection

### Imports & environment

In [18]:
import os
import random

import numpy as np
import matplotlib.pyplot as plt

from glob import iglob, glob
from models import Vgg16BN

from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img, img_to_array

from scipy.misc import imresize, toimage


ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR + '/data'
%matplotlib inline

### Config & Hyperparameters

In [19]:
# paths
data_path = DATA_HOME_DIR + '/fish_detection'
split_train_path = data_path + '/train/'
valid_path = data_path + '/valid/'
saved_model_path = ROOT_DIR + '/models/fish_detector_480x270/'

# data
batch_size = 8
nb_split_train_samples = 3277#3327
nb_full_train_samples = 3
nb_valid_samples = 500
nb_test_samples = 1000
classes = ["NoF", "FISH"]
nb_classes = len(classes)

# model
nb_runs = 1
nb_epoch = 35
aug = True
dropout = 0.3
lr=0.001
archs = ["vggbn"]

### Train the Fish Detector Model

In [20]:
def train():
    vgg = Vgg16BN(size=(270, 480), n_classes=nb_classes, lr=lr,
              batch_size=batch_size, dropout=dropout)

    vgg.build()
    model_fn = saved_model_path + '{val_loss:.2f}-loss_{epoch}epoch' +'_480x270_{0}-dropout_{1}-lr_vggbn.h5'.format(dropout, lr)
    ckpt = ModelCheckpoint(filepath=model_fn, monitor='val_loss',
                           save_best_only=True, save_weights_only=True)

    vgg.fit_val(split_train_path, valid_path, nb_trn_samples=nb_split_train_samples, 
                         nb_val_samples=nb_valid_samples, nb_epoch=nb_epoch, callbacks=[ckpt], aug=True)

    model_path = max(iglob(saved_model_path + '*.h5'), key=os.path.getctime)
    return model_path

model_path = train()

ValueError: Dimension 0 in both shapes must be equal, but are 61440 and 25088 for 'Assign_188' (op: 'Assign') with input shapes: [61440,4096], [25088,4096].

### Demo

In [21]:
vgg = Vgg16BN(size=(270, 480), n_classes=nb_classes, lr=lr, batch_size=batch_size, dropout=dropout)
vgg.build()

model = vgg.model
model.load_weights(saved_model_path + '0.03-loss_2epoch_480x270_0.3-dropout_0.001-lr_vggbn.h5')

ValueError: Dimension 0 in both shapes must be equal, but are 61440 and 25088 for 'Assign_215' (op: 'Assign') with input shapes: [61440,4096], [25088,4096].

In [14]:
valid_path

'/home/ipl/Downloads/0126/train/nc-fish-classification/nc_fish_classification/data/fish_detection/valid/'

In [15]:
sample_fish_fns = [fn for fn in glob(valid_path + '*/*.jpg') if "NoF" not in fn]
sample_nof_fns = [fn for fn in glob(valid_path + '*/*.jpg') if "NoF" in fn]

print("fish:", len(sample_fish_fns))
print("no fish:", len(sample_nof_fns))

fish: 435
no fish: 65


In [17]:
# Fish examples

ix = random.randint(0, len(sample_fish_fns))

im = imresize(load_img(sample_fish_fns[ix]), size=(270, 480))
test_x = img_to_array(im).reshape(1, 3, 270, 480)

pred = model.predict(test_x)[0]

print("Ground Truth: Fish\n")
print("Fish Prob:", pred[0])
print("NoFish Prob:", pred[1])

plt.imshow(im)

NameError: name 'model' is not defined

In [ ]:
# NoFish examples

ix = random.randint(0, len(sample_nof_fns))

im = imresize(load_img(sample_nof_fns[ix]), size=(270, 480))
test_x = img_to_array(im).reshape(1, 3, 270, 480)

pred = model.predict(test_x)[0]

print("Ground Truth: NoFish\n")
print("Fish Prob:", pred[0])
print("NoFish Prob:", pred[1])

plt.imshow(im)